<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Проект-для-&quot;Викишоп&quot;-с-BERT" data-toc-modified-id="Проект-для-&quot;Викишоп&quot;-с-BERT-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Проект для "Викишоп" с BERT</a></span><ul class="toc-item"><li><span><a href="#Вызовы-необходимых-библиотек" data-toc-modified-id="Вызовы-необходимых-библиотек-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вызовы необходимых библиотек</a></span></li></ul></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Лемматизация" data-toc-modified-id="Лемматизация-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Лемматизация</a></span></li><li><span><a href="#Разделение-на-обучающую-и-тестовую-выборки" data-toc-modified-id="Разделение-на-обучающую-и-тестовую-выборки-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Разделение на обучающую и тестовую выборки</a></span></li><li><span><a href="#Векторизация-с-помощью-TF-IDF" data-toc-modified-id="Векторизация-с-помощью-TF-IDF-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Векторизация с помощью TF-IDF</a></span></li></ul></li><li><span><a href="#Обучение-моделей-классификации" data-toc-modified-id="Обучение-моделей-классификации-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей классификации</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия-с-устранением-дисбаланса-классов" data-toc-modified-id="Логистическая-регрессия-с-устранением-дисбаланса-классов-3.1.1"><span class="toc-item-num">3.1.1&nbsp;&nbsp;</span>Логистическая регрессия с устранением дисбаланса классов</a></span></li></ul></li><li><span><a href="#XGBoost" data-toc-modified-id="XGBoost-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>XGBoost</a></span></li></ul></li><li><span><a href="#Решение-с-помощью-BERT" data-toc-modified-id="Решение-с-помощью-BERT-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Решение с помощью BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Проект для "Викишоп" с BERT

Интернет-магазин "Викишоп" запускает сервис, предоставляющий пользователям публиковать и дополнять описания товаров, комментировать изменения других пользователей. 

**Цель проекта** - разработать модель для поиска токсичных комментариев, чтобы можно было отправлять их на модерацию.

**Задачи проекта**:
* подготовить данные,
* обучить разные модели,
* добиться качества определения токсичных комментариев не менее 0,75 по F1.

Попробуем достичь цели проекта двумя способами - задействовав BERT и без этой нейросети.

## Вызовы необходимых библиотек

In [1]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
import xgboost as xgb

import torch
import transformers
from transformers import AutoTokenizer, AutoModel
from transformers import BertTokenizer, BertModel
from tqdm import notebook


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Мамуля\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Мамуля\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Подготовка данных

Откроем таблицу и посмотрим на данные.

In [2]:
try:
    data = pd.read_csv('C:\\Users\\Мамуля\\Documents\\toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')

data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
# содержится ли в столбце Unnamed: 0 какая-то значимая информация?
data['Unnamed: 0'].describe()

count    159292.000000
mean      79725.697242
std       46028.837471
min           0.000000
25%       39872.750000
50%       79721.500000
75%      119573.250000
max      159450.000000
Name: Unnamed: 0, dtype: float64

Итак, представлены комментарии на английском языке. В таблице также есть столбец "Unnamed:0", который не несёт какой-либо смысловой нагрузки (возможно, содержит старые индексы строк). Всего в таблице 159292 строк, все значения есть. Удалим лишний столбец и попробуем обнаружить дубликаты.

In [5]:
data = data.drop('Unnamed: 0', axis=1)

data.duplicated().sum()

0

Выборка для нашей задачи огромная - целесообразно взять произвольные наблюдения для каждого из подходов к решению задачи. Обозначим сэмплы df1 и df2: первый будем использовать для моделей, которые будут работать на данных, полученных с помощью TF-IDF, он будет включать 20000 строк. Второй - для BERT, пусть в нём будет 2000 строк (так как обработка большого массива текстов займёт очень много времени). 

In [6]:
df1 = data.sample(n=80000,random_state=12345).reset_index(drop=True)
df2 = data.sample(n=2000,random_state=12345).reset_index(drop=True)

# check
df2.shape

(2000, 2)

## Лемматизация

На первом этапе лемматизируем тексты комментариев - это потрубется для решения задачи любым их двух способов. Воспользуемся лемматизатором для английского языка - WordNetLemmatizer.

In [8]:
# создадим функцию, которая будет и лемматизировать, и токенизировать текст
lemmatizer = WordNetLemmatizer()

def lemmatize_and_clear(text):
    new_text = []
    for i in text:
        lem_text = " ".join(lemmatizer.lemmatize(word) for word in nltk.word_tokenize(i))
        clear_text = re.sub(r'[^a-zA-Z]', ' ', lem_text)
        new_text.append(" ".join(clear_text.split()))
    return new_text
    

In [9]:
# df_test = pd.DataFrame(['she likes these books...', 'wow! this is great', 'Peter goes to get some apples'], columns=['text'])

# df_test['lem_text'] = lemmatize_and_clear(df_test['text'])

# df_test

In [10]:
# применим функцию к обоим датасетам
df1['lem_text'] = lemmatize_and_clear(df1['text'])

df1.head()

,text,toxic,lem_text
0,Expert Categorizers \n\nWhy is there no menti...,0,Expert Categorizers Why is there no mention of...
1,"""\n\n Noise \n\nfart* talk. """,1,Noise fart talk
2,"An indefinite block is appropriate, even for a...",0,An indefinite block is appropriate even for a ...
3,I don't understand why we have a screenshot of...,0,I do n t understand why we have a screenshot o...
4,"Hello! Some of the people, places or things yo...",0,Hello Some of the people place or thing you ha...


In [11]:
df2['lem_text'] = lemmatize_and_clear(df2['text'])

Проблему могут представлять собой заглавные буквы в отдельных словах. Целесообразно перевести всё в нижний регистр.

In [12]:
df1['lem_text'] = df1['lem_text'].str.lower()
df2['lem_text'] = df2['lem_text'].str.lower()

df1.head()

,text,toxic,lem_text
0,Expert Categorizers \n\nWhy is there no menti...,0,expert categorizers why is there no mention of...
1,"""\n\n Noise \n\nfart* talk. """,1,noise fart talk
2,"An indefinite block is appropriate, even for a...",0,an indefinite block is appropriate even for a ...
3,I don't understand why we have a screenshot of...,0,i do n t understand why we have a screenshot o...
4,"Hello! Some of the people, places or things yo...",0,hello some of the people place or thing you ha...


Перед тем, как разделять данные на выборки для обучения моделей, посмотрим, нет ли дисбаланса классов - это важно для задачи классификации.

In [13]:
df1['toxic'].value_counts()

0    71909
1     8091
Name: toxic, dtype: int64

In [14]:
df2['toxic'].value_counts()

0    1786
1     214
Name: toxic, dtype: int64

In [15]:
print(df2['toxic'].value_counts()[1] / len(df2['toxic']))

0.107


Видим, что данные несбалансированы по классу, доля токсичных комментариев составляет примерно 1/10 от общего количества. Необходимо будет учесть это во время передачи параметров моделям.

## Разделение на обучающую и тестовую выборки

После первичной обработки разделим датасет 1 на обучающую и тестовую выборки, отделим целевой признак. Для датасета 2 эту операцию будем проводить позже - после получения признаков.

In [16]:
target1 = df1['toxic']
features1 = df1.drop(['toxic', 'text'], axis=1)

features_train1, features_test1, target_train1, target_test1, = train_test_split(
    features1, target1, test_size=0.2, random_state=12345)

# check
features_train1.shape

(64000, 1)

## Векторизация с помощью TF-IDF

Видим, что в лемматизированном тексте присутствуют артикли, отдельные буквы - эти частицы практически не несут какой-либо информации. Установим стоп-слова, чтобы избавиться от бесполезных частей текста, а затем применим TF-IDF.

In [17]:
stopwords = set(nltk_stopwords.words('english'))

In [18]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
# обучаем счётчик на обучающей выборке
# и применяем счётчик и к обучающей, и к тестовой выборке
features_train1_vec = count_tf_idf.fit_transform(features_train1['lem_text'])
features_train1_vec = count_tf_idf.transform(features_train1['lem_text'])
features_test1_vec = count_tf_idf.transform(features_test1['lem_text'])

# check
features_train1_vec.shape

(64000, 96347)

# Обучение моделей классификации

Попробуем выполнить классификацию с помощью двух моделей классификации: логистической регрессии и XGBoostClassifier. Помним о том, что наши данные не сбалансированы.

## Логистическая регрессия

Попробуем понять, какое качество по метрике F1 даст логистическая регрессия, если не предупредить модель о дисбалансе классов.

In [19]:
model_lr = LogisticRegression(random_state=12345)
model_lr.fit(features_train1_vec, target_train1)

LogisticRegression(random_state=12345)

In [20]:
lr_cv = cross_validate(model_lr, features_train1_vec, target_train1, cv=6, 
                     scoring='f1',
                    return_train_score=True)
lr_cv = pd.DataFrame(lr_cv)
display(lr_cv)
lr_cv['test_score'].mean()

,fit_time,score_time,test_score,train_score
0,1.445327,0.004001,0.653037,0.721642
1,1.402318,0.005001,0.666267,0.715865
2,0.940212,0.005001,0.650794,0.715932
3,1.759399,0.005002,0.653333,0.719015
4,1.267287,0.005001,0.668263,0.717579
5,1.536337,0.004001,0.685142,0.714753


0.6628059788660227

In [21]:
# model_lr = make_pipeline(TfidfVectorizer(stop_words=stopwords), 
                         # LogisticRegression(random_state=12345))

In [22]:
# scores_lr = cross_validate(model_lr, features_train1['lem_text'], target_train1, 
                        # scoring=['f1'], cv=3, return_train_score=True,
                          # error_score='raise')
# scores_lr = pd.DataFrame(scores_lr)
# lr_cv['test_score'].mean()

Без предупреждения о дисбалансе модель даёт низкий результат. Попробуем использовать встроенные параметры классификатора.

In [23]:
# пусть модель автоматически учтёт дисбаланс классов

model_lr = LogisticRegression(random_state=12345, max_iter=200, class_weight='balanced')
model_lr.fit(features_train1_vec, target_train1)

LogisticRegression(class_weight='balanced', max_iter=200, random_state=12345)

In [24]:
params = {'C': [2.8, 3, 3.2]}
grid = GridSearchCV(model_lr, params, cv=6, scoring='f1')
grid.fit(features_train1_vec, target_train1)

lr_scores = pd.DataFrame(grid.cv_results_)

display(lr_scores.head())
display(lr_scores[lr_scores['rank_test_score'] == 1])
# lr_cv = cross_validate(model_lr, params, features_train1_vec, target_train1, cv=3, 
                     # scoring='f1',
                    # return_train_score=True)
# lr_cv = pd.DataFrame(lr_cv)
# display(lr_cv)
# lr_cv.describe()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
0,1.955943,0.392814,0.004334,0.000472,2.8,{'C': 2.8},0.749234,0.755048,0.763975,0.741064,0.737335,0.760832,0.751248,0.009744,3
1,2.018457,0.249572,0.004834,0.000373,3,{'C': 3},0.751425,0.754070,0.762328,0.742133,0.740171,0.761285,0.751902,0.008512,2
2,1.966445,0.537161,0.004834,0.000373,3.2,{'C': 3.2},0.751648,0.754517,0.764235,0.743433,0.739503,0.764348,0.752947,0.009423,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,split5_test_score,mean_test_score,std_test_score,rank_test_score
2,1.966445,0.537161,0.004834,0.000373,3.2,{'C': 3.2},0.751648,0.754517,0.764235,0.743433,0.739503,0.764348,0.752947,0.009423,1


В среднем качество по метрике f1 на тренировочной выборке доходит до 0.75, что является неплохим результатом.

Протестируем модель на тестовой выборке и замерим метрику f1.

In [25]:
model_lr = LogisticRegression(random_state=12345, C=3.2, max_iter=200, class_weight='balanced')
model_lr.fit(features_train1_vec, target_train1)
predicted_lr = model_lr.predict(features_test1_vec)
print(f1_score(target_test1, predicted_lr))

0.7538959129667745


На тестовой выборке качество предсказаний модели достигло 0.75 (целевого уровня).

### Логистическая регрессия с устранением дисбаланса классов

Предыдущий опыт свидетельствует, что уменьшение выборки таким образом, чтобы классы были сбалансированы, ведёт к улучшению результатов по метрике f1. Попробуем добиться улучшения качества предсказаний логистической регрессии путём применения техники downsampling к df1.

Заново разделим набор данных на выборки.

In [26]:
target2 = data['toxic']
features2 = data.drop(['toxic'], axis=1)

# токенизируем и лемматизируем тексты в 
features2['lem_text'] = lemmatize_and_clear(features2['text'])

# теперь удалим столбец `text`
features2 = features2.drop(['text'], axis=1)

# разделяем обучающую и тестовую выборки
features_train, features_test2, target_train, target_test2, = train_test_split(
    features2, target2, test_size=0.2, random_state=12345)

In [27]:
# применим downsampling к обучающим признакам
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat([features_zeros.sample(frac=fraction, random_state=12345)] 
                                     + [features_ones])
    target_downsampled = pd.concat([target_zeros.sample(frac=fraction, random_state=12345)] 
                                   + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

features_train2, target_train2 = downsample(features_train, target_train, 0.1)

# check
features_train2.shape

(24404, 1)

Выборка получилась небольшой, но всё же проверим, будет ли улучшаться качество предсказаний.

In [28]:
features_train2_vec = count_tf_idf.fit_transform(features_train2['lem_text'])
features_train2_vec = count_tf_idf.transform(features_train2['lem_text'])
features_test2_vec = count_tf_idf.transform(features_test2['lem_text'])

# check
features_train2_vec.shape

(24404, 49228)

In [29]:
model_lr = LogisticRegression(random_state=12345, class_weight='balanced')
params = {'C': [2.8, 3.2, 3.4]}
grid = GridSearchCV(model_lr, params, cv=3, scoring='f1')
grid.fit(features_train2_vec, target_train2)

lr_scores = pd.DataFrame(grid.cv_results_)

display(lr_scores.head())
display(lr_scores[lr_scores['rank_test_score'] == 1])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.536121,0.071196,0.004001,4.495664e-07,2.8,{'C': 2.8},0.887186,0.884422,0.893449,0.888352,0.003777,3
1,0.596802,0.110056,0.004334,4.715390e-04,3.2,{'C': 3.2},0.887027,0.885516,0.893419,0.888654,0.003425,2
2,0.597802,0.032894,0.004001,8.165347e-04,3.4,{'C': 3.4},0.886975,0.885704,0.893632,0.888771,0.003477,1


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
2,0.597802,0.032894,0.004001,0.000817,3.4,{'C': 3.4},0.886975,0.885704,0.893632,0.888771,0.003477,1


Качество предсказаний на тестовой выборке заметно улучшилось - несмотря на скромные размеры самой выборки. Посмотрим, какое среднее гармоническое полноты и точности покажет модель на тестовой выборке.

In [30]:
model_lr = LogisticRegression(random_state=12345, C=3.4, class_weight='balanced')
model_lr.fit(features_train2_vec, target_train2)
predicted_lr2 = model_lr.predict(features_test2_vec)
print(f1_score(target_test2, predicted_lr2))

0.6741492537313433


C помощью выравнивания классов не удалось добиться улучшения качества предсказаний до целевого значения, но необходимо учитывать, что и выборка была небольшой.

## XGBoost

Интересно посмотреть, какое качество предсказаний даст модель на основе градиентного бустинга. XGBoost предоставляет возможность задать информацию о дисбалансе классов с помощью параметра `scale_pos_weight`, значение которого должно равняться результату деления количества наблюдений "0" на количество "1".

In [31]:
model_xgb = xgb.XGBClassifier(random_state=12345, scale_pos_weight=9)

params = {'n_estimators': range (300, 500, 100),
              'max_depth': [4, 5, 6],
         'learning_rate': [0.4, 0.3]}
grid = GridSearchCV(model_xgb, params, cv=5, scoring='f1')
grid.fit(features_train1_vec, target_train1)

[15:30:33] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:30:50] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:31:06] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:53:11] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:53:42] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=12345, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weigh

In [32]:
xgb_scores = pd.DataFrame(grid.cv_results_)

display(xgb_scores.head())
display(xgb_scores[xgb_scores['rank_test_score'] == 1])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,16.543348,0.452326,0.049011,0.000632,0.4,4,300,"{'learning_rate': 0.4, 'max_depth': 4, 'n_esti...",0.731529,0.758696,0.744807,0.741168,0.748170,0.744874,0.008874,11
1,21.900962,0.257044,0.055813,0.004309,0.4,4,400,"{'learning_rate': 0.4, 'max_depth': 4, 'n_esti...",0.745926,0.765459,0.746925,0.748092,0.753420,0.751964,0.007226,7
2,20.205378,0.252786,0.053412,0.002871,0.4,5,300,"{'learning_rate': 0.4, 'max_depth': 5, 'n_esti...",0.747768,0.758595,0.736449,0.747985,0.758801,0.749920,0.008294,8
3,26.311161,0.407898,0.056813,0.000980,0.4,5,400,"{'learning_rate': 0.4, 'max_depth': 5, 'n_esti...",0.756226,0.763868,0.741497,0.760178,0.753776,0.755109,0.007625,3
4,23.277473,0.100329,0.057813,0.005777,0.4,6,300,"{'learning_rate': 0.4, 'max_depth': 6, 'n_esti...",0.751683,0.760238,0.747713,0.755970,0.753711,0.753863,0.004188,4


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,30.938609,0.066101,0.061614,0.003775,0.3,6,400,"{'learning_rate': 0.3, 'max_depth': 6, 'n_esti...",0.748587,0.768715,0.751327,0.759239,0.759819,0.757537,0.007095,1


Таким образом, на обучающей выборке модель XGBoost добилась максимального качества предсказаний около 0.758 по метрике f1. Обучим модель с оптимальными параметрами и посмотрим, каково будет качество предсказаний на тестовой выборке.

In [33]:
model_xgb = xgb.XGBClassifier(learning_rate=0.3, max_depth=6, 
                              n_estimators=400, scale_pos_weight=9, 
                              random_state=12345)

model_xgb.fit(features_train1_vec, target_train1)
predicted_xgb = model_xgb.predict(features_test1_vec)

print(f1_score(target_test1, predicted_xgb))

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:54:21] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.7434491163924437


На тестовой выборке качество предсказаний снизилось - до уровня 0.74 по f1. 

Теперь интересно посмотреть, как модель XGB будет работать на вручную сбалансированном по классу датасете.

In [34]:
model_xgb = xgb.XGBClassifier(random_state=12345)

params = {'n_estimators': range (100, 400, 100),
              'max_depth': [3, 4, 5],
         'learning_rate': [0.3, 0.2, 0.1]}
grid = GridSearchCV(model_xgb, params, cv=3, scoring='f1')
grid.fit(features_train2_vec, target_train2)

[15:55:01] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:55:02] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:55:03] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


...

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:59:21] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:59:27] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None,
                                     enable_categorical=False, gamma=None,
                                     gpu_id=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None, max_delta_step=None,
                                     max_depth=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     n_estimators=100, n_jobs=None,
                                     num_parallel_tree=None, predictor=None,
                                     random_state=12345, reg_alpha=None,
                                     reg_lambda=None, scale_pos_weigh

In [35]:
xgb_scores = pd.DataFrame(grid.cv_results_)

display(xgb_scores[xgb_scores['rank_test_score'] == 1])

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
8,5.72156,0.061389,0.032819,0.003775,0.3,5,300,"{'learning_rate': 0.3, 'max_depth': 5, 'n_esti...",0.884611,0.882396,0.88517,0.884059,0.001198,1


На сбалансированном датасете результаты вновь намного лучше. Попробуем обучить модель с наилучшими параметрами и получить предсказания с её помощью.

In [36]:
model_xgb = xgb.XGBClassifier(learning_rate=0.3, max_depth=5, 
                              n_estimators=300, random_state=12345)

model_xgb.fit(features_train2_vec, target_train2)
predicted_xgb = model_xgb.predict(features_test2_vec)

print(f1_score(target_test2, predicted_xgb))

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[15:59:36] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6890694239290989


В "тепличных" условиях классификатор XGBoost показывает умеренный уровень качества в определении тональности комментария даже без учёта контекста, добиться целевого уровня качества этой модели не удалось.

# Решение с помощью BERT

Для второго пути решения задачи возьмём второй сэмпл - `df2`, состоящий из 2000 значений - и уже обученную модель Conversational BERT (English, cased, 12-layer, 768-hidden, 12-heads, 110M parameters). После того, как модель BERT векторизует текст, будем использовать для классификации логистическую регрессию.

У BERT также есть собственные средства обработки текста. Воспользуемся этим инструментарием.

In [37]:
# tokenizer = transformers.BertTokenizer(vocab_file='C:\\Users\\Мамуля\\Documents\\vocab.txt')

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# токенизируем столбец
# tokenized = df2['lem_text'].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

# найдём максимальную длину вектора
# max_len = 0
# for i in tokenized.values:
    # if len(i) > max_len:
        # max_len = len(i)

# применим padding
# padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создадим маску
# attention_mask = np.where(padded != 0, 1, 0)

# check 
# attention_mask.shape

Получается, самый длинный комментарий состоит из 1720 слов - объём эссе. Выбранная предобученная модель BERT не работает с текстами такой длины (максимум - 512 слов). Поэтому целесообразно переписать приведённый выше код с учётом этой особенности: на этапе токенизации предложить максимальную длину, допустим, 500 слов. В этом случае длинные комментарии придётся обрезать, но, будем надеяться, что это не очень значительно повлияет на качество предсказаний впоследствии.

In [38]:
# токенизируем столбец c заданной длиной
tokenized = df2['lem_text'].apply(lambda x: tokenizer.encode(x, max_length=500, 
                                                             add_special_tokens=True, 
                                                             truncation=True))

# найдём максимальную длину вектора
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

# применим padding
padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values])

# создадим маску
attention_mask = np.where(padded != 0, 1, 0)

# check 
attention_mask.shape

(2000, 500)

In [39]:
# инициализируем конфигурацию
# config = transformers.BertConfig.from_json_file(
    #'C:\\Users\\Мамуля\\Documents\\config.json')

# инициализируем саму модель
# model = transformers.BertModel.from_pretrained(
    # 'C:\\Users\\Мамуля\\Documents\\pytorch_model.bin', config=config, ignore_mismatched_sizes=True)

model = AutoModel.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Далее пропишем цикл для преобразования текстов в эмбеддинги.

In [40]:
batch_size = 100

embeddings = []

for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:].numpy())


  0%|          | 0/20 [00:00<?, ?it/s]

In [41]:
# создадим признаки и целевой признак
features = np.concatenate(embeddings)
target = df2['toxic']

# проведём разделение на выборки, пусть в тестовой будет 500 строк
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

Протестируем, как на эмбеддингах будет работать логистическая регрессия. Предупредим модель о дисбалансе классов.

In [42]:
model_b = LogisticRegression(random_state=12345, class_weight='balanced')
model_b.fit(features_train, target_train)
predicted = model_b.predict(features_test)

print(f1_score(target_test, predicted))

0.6875


C:\Python\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Оказалось, что качество предсказания на тестовой выборке несколько ниже, чем у логистической регрессии, обученной на признаках TF-IDF.

Посмотрим, будет ли ситуация с градиентным бустингом лучше.

In [43]:
model_xgb_2 = xgb.XGBClassifier(random_state=12345, scale_pos_weight=9)

params = {'n_estimators': range (100, 500, 100),
              'max_depth': [3, 4, 5],
         'learning_rate': [0.3, 0.2, 0.1]}
grid = GridSearchCV(model_xgb_2, params, cv=3, scoring='f1')
grid.fit(features_train, target_train)

xgb_scores = pd.DataFrame(grid.cv_results_)

display(xgb_scores[xgb_scores['rank_test_score'] == 1])

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:48:51] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:48:51] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


...

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:32] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:34] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
16,0.969219,0.039833,0.006002,2.247832e-07,0.2,4,100,"{'learning_rate': 0.2, 'max_depth': 4, 'n_esti...",0.716981,0.575,0.554217,0.615399,0.072329,1


In [44]:
model_xgb_2 = xgb.XGBClassifier(learning_rate=0.2, max_depth=4, 
                              n_estimators=100, random_state=12345)

model_xgb_2.fit(features_train, target_train)
predicted_xgb_2 = model_xgb_2.predict(features_test)

print(f1_score(target_test, predicted_xgb_2))

C:\Python\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[16:51:36] WARNING: C:\Windows\Temp\abs_557yfx631l\croots\recipe\xgboost-split_1659548953302\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6341463414634146


XGB Classifier также хуже справляется с предсказанием.

# Выводы

Для решения задачи (выявления токсичных комментариев) применялись два подхода:
* Токенизация текстов комментариев с помощью TF-IDF и затем классификация с помощью линейной регрессии и классификатора XGBoost,
* Токенизация текстов с помощью предобученной модели BERT, создание эмбеддингов и затем классификация с помощью тех же двух моделей.

Для работы по первому сценарию из общего массива данных был выбран сэмпл размером 80000 строк, для второго сценария – 2000 строк.

В данном случае значительной проблемой стала несбалансированность классов: естественно, токсичные комментарии появляются сравнительно редко: в исследуемом случае их было всего около 10% от общего количества.

В рамках первого сценария удалось добиться порогового уровня качества по метрике f1 при использовании логистической регрессии - для этого модели необходимо было указать на наличие дисбаланса классов, а также подобрать оптимальное значение силы регуляризации. После устранения дисбаланса с помощью downsampling качество предсказаний снизилось у обеих моделей. Большое влияние на качество оказывает размер выборки.

В рамках второго сценария ещё одной проблемой стала длина комментариев: выбранная предобученная модель BERT может работать с текстами не длиннее 512 слов, тогда как в данных попадались наблюдения с гораздо большим количеством слов. Решено было ограничить максимальную длину токенов 500. 
По всей видимости, дисбаланс классов сказался на результатах работы моделей и в случае эмбеддингов: ни одной из двух моделей классификации не удалось показать качество выше, чем в случае использования признаков TF-IDF.
